In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
from core.interpolator import Interpolator

In [3]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [ ]:
import os
from dotenv import load_dotenv
env_path = "templates/app_schema/.env"
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"Loaded environment variables from {env_path}")

Loaded environment variables from templates/app_schema/.env


In [5]:
application = Application(client, compiler)
interpolator = Interpolator(".")

In [6]:
user_prompts = """Bot that says hello""".strip()

In [7]:

my_bot = application.prepare_bot([user_prompts], bot_id="handler_tests", capabilities=["perplexity.web_search"])
my_bot.__dict__

Compiling TypeSpec...


{'refined_description': None,
 'capabilities': CapabilitiesOut(capabilities=['perplexity.web_search'], error_output=None),
 'typespec': TypespecOut(reasoning="This is a simple bot that greets users. Let me think about what a hello bot might need to do:\n\n1. Say hello to a user\n2. Maybe personalize the greeting with the user's name\n3. Potentially customize the greeting based on time of day or other parameters\n\nFunctions I can identify from the description:\n- greetUser: To say hello to a user, possibly with personalization options\n- customizeGreeting: To set preferences for how greetings are delivered\n\nFor the data models:\n- UserGreeting: For greeting a user, containing things like name, custom message, etc.\n- GreetingPreferences: For setting preferences like greeting style, formality, etc.", typespec_definitions='model UserGreeting {\n  name?: string;\n  customMessage?: string;\n  timeOfDay?: string;\n}\n\nmodel GreetingPreferences {\n  style?: string;\n  formality?: string;\

In [8]:
my_bot = application.prepare_bot([user_prompts, "Add say goodbye"], bot_id="handler_tests", capabilities=["perplexity.web_search"])
my_bot.__dict__

Compiling TypeSpec...


{'refined_description': None,
 'capabilities': CapabilitiesOut(capabilities=['perplexity.web_search'], error_output=None),
 'typespec': TypespecOut(reasoning="Based on the user requests, I need to create a simple greeting bot that can say hello and goodbye. The application doesn't seem to require complex data structures, but we'll need to formalize the interactions.\n\nThe key functions needed are:\n1. sayHello - A function to greet the user\n2. sayGoodbye - A function to say goodbye to the user\n\nFor the arguments, we should keep it simple:\n- For sayHello, we might want the user's name as an option\n- For sayGoodbye, we might want a reason for leaving or similar contextual information\n\nEach function needs proper @llm_func descriptions and @scenario decorators with Gherkin syntax to describe the expected behavior.", typespec_definitions='model GreetingOptions {\n    name: string;\n    timeOfDay?: string;\n}\n\nmodel FarewellOptions {\n    name: string;\n    reason?: string;\n}\n\ni

In [9]:
my_bot = application.update_bot(
    typespecSchema=my_bot.typespec.typespec_definitions, 
    bot_id="web_search_bot", 
    capabilities=["perplexity.web_search"]
)

TypeError: Application.update_bot() missing 1 required positional argument: 'typespecSchema'

In [10]:
interpolator.bake(my_bot, "../../bot_result") # next to repo root

In [11]:
my_bot.__dict__

{'refined_description': RefineOut(refined_description='model HelloOptions {\n    userName?: string;\n    timeOfDay?: string;\n}\n\nmodel GoodbyeOptions {\n    userName?: string;\n    reasonForLeaving?: string;\n}\n\ninterface GreetingBot {\n    @scenario(\n    """\n    Scenario: Basic greeting with name\n    When user says "Hello"\n    Then system responds with greeting\n    \n    Scenario: Personalized greeting with name\n    When user says "Hi my name is John"\n    Then system responds with personalized greeting using name "John"\n    \n    Scenario: Time-specific greeting\n    When user says "Good morning"\n    Then system responds with morning-appropriate greeting\n    \n    Examples:\n        | Input                   | Expected Behavior           |\n        | "Hi"                    | Generic greeting            |\n        | "Hello, I\'m Sarah"      | Greeting with name "Sarah"  |\n        | "Good evening"          | Evening-appropriate greeting|\n    """)\n    @llm_func("Generat